In [1]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

keras.set_image_data_format('channels_last')

def unet(pretrained_weights = None,input_size = (2800,2400,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


Using TensorFlow backend.


In [9]:
fnames = ["/Users/cdesio/UoB/Jordan/10x10_20mm_v2/File_{}_25mm_mask_*.tiff".format(i) for i in range(100)]

In [3]:
#fnames_orig = ["/Users/cdesio/UoB/Jordan/10x10_25mm_v1/{}".format(filename) 
#               for filename in sorted(os.listdir("/Users/cdesio/UoB/Jordan/10x10_25mm_v1/")) if "mask" not in filename and filename.startswith('File')]
fnames_orig = ["/Users/yw18581/physics/Jordan/10x10_25mm_v1/{}".format(filename) 
               for filename in sorted(os.listdir("/Users/yw18581/physics/Jordan/10x10_25mm_v1/")) if "mask" not in filename and filename.startswith('File')]

In [4]:
#fnames_mask = ["/Users/cdesio/UoB/Jordan/10x10_25mm_v1/{}".format(filename) 
#               for filename in sorted(os.listdir("/Users/cdesio/UoB/Jordan/10x10_25mm_v1/")) if "mask" in filename and filename.startswith('File')]
fnames_mask = ["/Users/yw18581/physics/Jordan/10x10_25mm_v1/{}".format(filename) 
               for filename in sorted(os.listdir("/Users/yw18581/physics/Jordan/10x10_25mm_v1/")) if "mask" in filename and filename.startswith('File')]

In [5]:
fnames_mask_train = fnames_mask[:10]
fnames_orig_train = fnames_orig[:10]

In [6]:
from matplotlib.image import imread


In [7]:
X = np.asarray([imread(img) for img in fnames_orig_train])
y = np.asarray([imread(img) for img in fnames_mask_train])

In [8]:
X.shape

(10, 2800, 2400)

In [9]:
Xr = X.reshape(10,2800,2400,1)

In [10]:
Xr.shape

(10, 2800, 2400, 1)

In [11]:
y.shape

(10, 2800, 2400)

In [12]:
yr = y.reshape(10,2800,2400,1)

In [13]:
yr.shape

(10, 2800, 2400, 1)

In [ ]:
model = unet()

model.fit(x=Xr, y=yr, epochs=1, verbose=1)
#testGene = testGenerator("data/membrane/test")
#results = model.predict_generator(testGene,30,verbose=1)
#saveResult("data/membrane/test",results)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2800, 2400, 1 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 2800, 2400, 6 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 2800, 2400, 6 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxP

/Users/yw18581/env/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/1


In [78]:
from keras import backend as K
print(K.image_data_format()) # print current format
#K.set_image_data_format('channels_last')

channels_first
